In [112]:
# TODO
# catologue risks
# refer to hse (health and safety executive)
# We have caught carp, barbel, roach, rudd, perch, skimmer, bream, gudgeon, tench,
#

In [113]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [114]:
from typing import Literal
from pydantic import BaseModel, Field


class Risk(BaseModel):
    """Describes a risk."""

    hazard: str = Field(
        description=(
            "Concise description of the hazard. "
            "A hazard is something that may cause harm or damage."
        )
    )
    risks: list[str] = Field(
        description=(
            "Risks associated with the hazard. "
            "A risk is the harm that may occur from the hazard. "
            "Describe each risk in a sentence."
        )
    )
    affects: Literal["Young people", "Leaders", "All present"] = Field(
        description="Who is at risk. Leaders are adult Scout leaders."
    )
    controls: list[str] = Field(
        description=(
            "Controls are ways of making the activity safer by removing or reducing the "
            "risk. For example, you may use a different piece of equipment or you might "
            "change the way you do the activity."
        )
    )


class Risks(BaseModel):
    """List of risks."""

    risks: list[Risk]

In [115]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate


system = """
A scout leader planning an activity for Scouts (aged 10-14) needs to perform a 
risk assessment for an activity at a given location.

Your task is to perform a risk assessment identifying hazards and the risks
associated with them.

When identifying hazards and risks, pay attention to the location as this can 
introduce additional hazards.

Always consider the impact that poor behaviour of young people could have.

Use plain British english and commonly used terms in your response.
"""

examples = []

user_prompt = """
Location: {location}

Activity: {activity}

Identify {n} risk(s). Rank them from highest impact to lowest impact.
"""

examples = [
    HumanMessage(
        user_prompt.format(
            location="The scout hut is large brick building with a hard wooden floor and a raised stage at the front. I contains a number of pillars to support the roof and stacked chairs around the edges of the hall.",
            activity="Variety of indoor games such as dodgeball, crab football etc.",
            n=1,
        )
    ),
    AIMessage(
        content="",
        name="call_01",
        tool_calls=[
            {
                "name": "risks",
                "args": {
                    "risks": [
                        {
                            "hazard": "Use of hot surfaces such as the stove or frying pan.",
                            "risks": [
                                "Scouts can get burned if they touch hot surfaces or spill hot oil.",
                                "Severe burns may require medical attention.",
                            ],
                            "affects": "Young people",
                            "controls": ["Explain and demonstrate how to use the cooking equipment"]
                        },
                    ]
                },
                "id": "1",
            }
        ],
    ),
    # Most tool-calling models expect a ToolMessage(s) to follow an AIMessage with tool calls.
    ToolMessage("", tool_call_id="01"),
    # Some models also expect an AIMessage to follow any ToolMessages,
    # so you may need to add an AIMessage here.
]

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("placeholder", "{examples}"), ("human", user_prompt)]
)

In [116]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
llm = model.with_structured_output(Risks, include_raw=True)
app = prompt | llm

risks = app.invoke(
    {
        # "activity": "Pumpkin carving with tee light candles. Pumpkin carving will be done on tables in groups of 2, 2 groups per table.",
        "activity": "Variety of indoor games such as dodgeball, crab football etc.",
        # "activity": "Scouts making pancakes in the kitchen when being supervised by 2 adults.",
        # "location": "The scout kitchen is a medium sized kitchen similar to a typical family kitchen",
        "location": "The scout hut is large brick building with a hard wooden floor and a raised stage at the front. I contains a number of exposed pillars to support the roof and stacked chairs.",
        "n": 7,
    }
)

# TODO would generally run this generation several times and filter

In [117]:
print(risks["parsed"].model_dump_json(indent=3))

{
   "risks": [
      {
         "hazard": "Exposed pillars in the scout hut",
         "risks": [
            "Young people may run into the pillars and sustain injuries such as bruises or cuts.",
            "Pillars may obstruct visibility during games, leading to collisions."
         ],
         "affects": "Young people",
         "controls": [
            "Ensure a clear playing area by marking boundaries away from pillars.",
            "Supervise games closely to prevent rough play near pillars."
         ]
      },
      {
         "hazard": "Hard wooden floor",
         "risks": [
            "Young people may slip and fall, leading to sprains or fractures.",
            "Inadequate footwear may increase the risk of slipping. "
         ],
         "affects": "All present",
         "controls": [
            "Ensure all participants wear appropriate footwear with good grip.",
            "Conduct a safety briefing on how to move safely on the floor."
         ]
      },
     

In [118]:
# FLOW
# identify risks -> generate x3 -> combine similar risks